In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from tensorflow import keras

def prepare_data(seq,num):
  x=[]
  y=[]
  for i in range(0,(len(seq)-num),1):
    
    input_ = seq[i:i+num]
    output  = seq[i+num]
    
    x.append(input_)
    y.append(output)
    
  return np.array(x), np.array(y)

data=pd.read_csv('../Utilities\csv\metrics_2024-09-05\metrics.csv')
print(data.head)
cpu_usage = data['value'].values

num=40 # The first 10 mins of traffic
sample = cpu_usage[:num]

x,y= prepare_data(cpu_usage,num)
print(len(x))

ind = int(0.9 * len(x))
x_tr = x[:ind]
y_tr = y[:ind]
x_val=x[ind:]
y_val=y[ind:]

#normalize the inputs
x_scaler= StandardScaler()
x_tr = x_scaler.fit_transform(x_tr)
x_val= x_scaler.transform(x_val)

#reshaping the output for normalization
y_tr=y_tr.reshape(len(y_tr),1)
y_val=y_val.reshape(len(y_val),1)

#normalize the output
y_scaler=StandardScaler()
y_tr = y_scaler.fit_transform(y_tr)[:,0]
y_val = y_scaler.transform(y_val)[:,0]

#reshaping input data
x_tr= x_tr.reshape(x_tr.shape[0],x_tr.shape[1],1)
x_val= x_val.reshape(x_val.shape[0],x_val.shape[1],1)
print(x_tr.shape)

def traffic_prediction_lstm():
    model = Sequential()
    model.add(LSTM(80, return_sequences=True, recurrent_activation='relu', input_shape=(num,1)))
    model.add(LSTM(60, return_sequences=True, recurrent_activation='relu', input_shape=(num,1)))
    model.add(LSTM(40, return_sequences=True, recurrent_activation='relu', input_shape=(num,1)))
    model.add(LSTM(20, return_sequences=True, recurrent_activation='relu', input_shape=(num,1)))
    model.add(LSTM(10,  dropout=0.2)) 
    model.add(Dense(1,activation='relu'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

lstm_model = traffic_prediction_lstm()
es = EarlyStopping(monitor="val_loss", patience=2)
lstm_training = lstm_model.fit(x_tr, y_tr, epochs=30, verbose=True, validation_split=0.1, batch_size=100, callbacks=[es])
# define model
# model =  Sequential()
# model.add(Conv1D(64, 3, padding='same', activation='relu',input_shape=(num,1)))
# model.add(Conv1D(32, 5, padding='same', activation='relu',input_shape=(num,1)))
# model.add(Flatten())
# model.add(Dense(64,activation='relu'))
# model.add(Dense(1,activation='linear'))
# model.summary()
# # Define the optimizer and loss:
# model.compile(loss='mse',optimizer='adam')
# # Define the callback to save the best model during the training
# mc = ModelCheckpoint('best_model.hdf5', monitor='val_loss', verbose=1,
#          save_best_only=True, mode='min')
# # Train the model for 30 epochs with batch size of 32:
# history=model.fit(x_tr, y_tr ,epochs=30, batch_size=32, validation_data=(x_val,y_val),
#             callbacks=[mc])
# model.load_weights('best_model.hdf5')
# mse = model.evaluate(x_val,y_val)
# print("Mean Square Error:",mse)

<bound method NDFrame.head of               timestamp         value
0   05/09/2024 12:00:00  3.186201e-05
1   05/09/2024 12:01:00  6.300736e-05
2   05/09/2024 12:02:00  2.806237e-05
3   05/09/2024 12:03:00  6.507017e-05
4   05/09/2024 12:04:00  6.088904e-05
..                  ...           ...
94  05/09/2024 13:34:00  6.124009e-05
95  05/09/2024 13:35:00  3.326552e-05
96  05/09/2024 13:36:00  3.272567e-05
97  05/09/2024 13:37:00  3.249500e-05
98  05/09/2024 13:38:00  3.470863e-07

[99 rows x 2 columns]>
59
(53, 40, 1)
